In [3]:
import pandas as pd
import numpy as np
import glob
import os
import itertools
from datetime import datetime
from pathlib import Path

# ==========================================
# 1. PROJECT CONFIGURATION
# ==========================================

# PATH TO YOUR DATA
CURRENT_DIR = Path.cwd().parent
DATA_PATH = CURRENT_DIR / "data"

# ASSET SPECIFICATIONS (Group 2)
ASSETS_CONFIG = {
    'XAU': {'cost': 15.0, 'point_value': 100.0},  # Gold
    'XAG': {'cost': 10.0, 'point_value': 5000.0}  # Silver
}

# IN-SAMPLE QUARTERS
IN_SAMPLE_QUARTERS = [
    ('2023-01-01', '2023-03-31'), # 2023 Q1
    ('2023-07-01', '2023-09-30'), # 2023 Q3
    ('2023-10-01', '2023-12-31'), # 2023 Q4
    ('2024-04-01', '2024-06-30'), # 2024 Q2
    ('2024-10-01', '2024-12-31'), # 2024 Q4
    ('2025-01-01', '2025-03-31'), # 2025 Q1
    ('2025-04-01', '2025-06-30'), # 2025 Q2
]

# ==========================================
# 2. EXPANDED PARAMETER GRIDS
# ==========================================

# Session Modes: 'full' (All Day), 'us_only' (13:00-16:30 CET)
SESSION_MODES = ['full', 'us_only']

# STRATEGY 1: VOLATILITY BREAKOUT
GRID_S1_XAU = {
    'window': [24, 72, 144, 288],      # 2h, 6h, 12h, 24h
    'vol_threshold': [0.5, 1.0, 1.5],  # Gold volatility
    'session': SESSION_MODES
}

GRID_S1_XAG = {
    'window': [48, 96, 144, 288],      # Silver needs wider windows
    'vol_threshold': [0.03, 0.05, 0.08],
    'session': SESSION_MODES
}

# STRATEGY 2: EMA + RSI (Generated Combinations)
GRID_S2_LIST = []
ema_pairs = [(10, 50), (20, 80), (50, 200)]
rsi_setups = [{'max': 70, 'min': 30}, {'max': 80, 'min': 20}]

for fast, slow in ema_pairs:
    for rsi in rsi_setups:
        for sess in SESSION_MODES:
            GRID_S2_LIST.append({
                'fast': fast, 'slow': slow,
                'rsi_max': rsi['max'], 'rsi_min': rsi['min'],
                'session': sess
            })

# ==========================================
# 3. HELPER FUNCTIONS
# ==========================================

def calculate_rsi(series, period=14):
    """Calculates RSI on Close prices."""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    avg_gain = gain.ewm(com=period - 1, min_periods=period).mean()
    avg_loss = loss.ewm(com=period - 1, min_periods=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(50).values

# ==========================================
# 4. DATA LOADING (UTC -> CET)
# ==========================================

def load_data_group2(data_folder):
    """Loads parquet files, handles Index, converts UTC to CET."""
    files = sorted(glob.glob(os.path.join(data_folder, "data2_*.parquet")))

    if not files:
        print(f"CRITICAL ERROR: No files found in {data_folder}")
        return pd.DataFrame()

    print(f"Loading {len(files)} files...")
    df_list = []

    for f in files:
        try:
            q_df = pd.read_parquet(f)
            if 'datetime' in q_df.columns:
                q_df['datetime'] = pd.to_datetime(q_df['datetime'])
                q_df.set_index('datetime', inplace=True)

            # UTC to Europe/Berlin (CET)
            if q_df.index.tz is None:
                q_df.index = q_df.index.tz_localize('UTC')
            q_df.index = q_df.index.tz_convert('Europe/Berlin')

            df_list.append(q_df)
        except Exception as e:
            print(f"Error loading {f}: {e}")

    if not df_list: return pd.DataFrame()

    full_df = pd.concat(df_list).sort_index()
    return full_df[~full_df.index.duplicated(keep='first')]

# ==========================================
# 5. STRATEGY ENGINE (WITH TIME FILTER)
# ==========================================

def run_strategy_engine(df_wide, asset_name, params, asset_conf, strat_type):
    if asset_name not in df_wide.columns: return None

    df = df_wide[[asset_name]].copy()
    df.columns = ['close']
    df.dropna(inplace=True)

    # Time Filters
    times = df.index.time
    t_vals = np.array([t.hour * 100 + t.minute for t in times])

    position = np.zeros(len(df))
    close = df['close'].values
    current_pos = 0

    # SESSION FILTER SETUP
    # 'full' = trade anytime allowed
    # 'us_only' = only ENTER between 13:00 and 16:30
    session_mode = params.get('session', 'full')
    entry_start = 1300 if session_mode == 'us_only' else 0
    entry_end = 1630 if session_mode == 'us_only' else 1650

    # --- LOGIC SELECTION ---
    if strat_type == 'breakout':
        window = params['window']
        vol_win = 20
        vol_th = params['vol_threshold']

        roll_max = df['close'].rolling(window).max().shift(1).values
        roll_min = df['close'].rolling(window).min().shift(1).values
        vol = df['close'].diff().abs().rolling(vol_win).mean().values

        for i in range(1, len(df)):
            t = t_vals[i]

            # 1. STRICT EXIT 16:50
            if 1650 <= t < 1810:
                current_pos = 0
                position[i] = 0
                continue

            # 2. SESSION CHECK (For Entry)
            can_enter = True
            if session_mode == 'us_only':
                if not (entry_start <= t < entry_end):
                    can_enter = False

            # STRATEGY
            if current_pos == 0:
                if can_enter and vol[i] > vol_th:
                    if close[i] > roll_max[i]: current_pos = 1
                    elif close[i] < roll_min[i]: current_pos = -1
            elif current_pos == 1:
                if close[i] < roll_min[i]: current_pos = -1
            elif current_pos == -1:
                if close[i] > roll_max[i]: current_pos = 1

            position[i] = current_pos

    elif strat_type == 'ema_rsi':
        fast = params['fast']
        slow = params['slow']
        rsi_vals = calculate_rsi(df['close'], 14)
        rsi_max = params['rsi_max']
        rsi_min = params['rsi_min']

        ema_f = df['close'].ewm(span=fast, adjust=False).mean().values
        ema_s = df['close'].ewm(span=slow, adjust=False).mean().values

        for i in range(1, len(df)):
            t = t_vals[i]

            # 1. STRICT EXIT 16:50
            if 1650 <= t < 1810:
                current_pos = 0
                position[i] = 0
                continue

            # 2. SESSION CHECK
            can_enter = True
            if session_mode == 'us_only':
                if not (entry_start <= t < entry_end):
                    can_enter = False

            # STRATEGY
            if ema_f[i] > ema_s[i]: # Bullish
                if current_pos != 1 and can_enter and rsi_vals[i] < rsi_max:
                    current_pos = 1
                elif current_pos == -1: current_pos = 0

            elif ema_f[i] < ema_s[i]: # Bearish
                if current_pos != -1 and can_enter and rsi_vals[i] > rsi_min:
                    current_pos = -1
                elif current_pos == 1: current_pos = 0

            position[i] = current_pos

    # PnL Calculation
    df['position'] = position
    df['price_change'] = df['close'].diff()
    df['gross_pnl'] = df['position'].shift(1) * df['price_change'] * asset_conf['point_value']
    df['trades'] = df['position'].diff().abs()
    df['tx_cost'] = df['trades'] * asset_conf['cost']
    df['net_pnl'] = df['gross_pnl'] - df['tx_cost']

    daily = df.resample('D').agg({'gross_pnl': 'sum', 'net_pnl': 'sum', 'trades': 'sum'}).fillna(0)
    daily.index = daily.index.tz_localize(None)
    return daily

# ==========================================
# 6. SCORING & OPTIMIZATION
# ==========================================

def calculate_quarter_stats(daily_df, q_start, q_end):
    mask = (daily_df.index >= q_start) & (daily_df.index <= q_end)
    sub = daily_df.loc[mask]
    if len(sub) == 0: return None

    gross_cum = sub['gross_pnl'].sum()
    net_cum = sub['net_pnl'].sum()
    avg_trades = sub['trades'].mean()

    std_net = sub['net_pnl'].std()
    std_gross = sub['gross_pnl'].std()
    net_sr = (sub['net_pnl'].mean() / std_net * np.sqrt(252)) if std_net > 1e-9 else 0
    gross_sr = (sub['gross_pnl'].mean() / std_gross * np.sqrt(252)) if std_gross > 1e-9 else 0

    days = (pd.Timestamp(q_end) - pd.Timestamp(q_start)).days
    frac_year = max(days / 365.25, 0.2)

    cum_net = sub['net_pnl'].cumsum()
    max_dd = (cum_net - cum_net.expanding().max()).min()
    net_cr = ((net_cum/frac_year) / abs(max_dd)) if max_dd != 0 else 0

    cum_gross = sub['gross_pnl'].cumsum()
    max_dd_g = (cum_gross - cum_gross.expanding().max()).min()
    gross_cr = ((gross_cum/frac_year) / abs(max_dd_g)) if max_dd_g != 0 else 0

    try:
        log_term = np.log(abs(net_cum / 1000.0)) if abs(net_cum) >= 1 else 0
        stat = (net_sr - 0.5) * max(0, log_term)
    except:
        stat = 0

    return {
        'Quarter': f"{q_start[:7]}",
        'Net_SR': net_sr, 'Net_PnL': net_cum, 'Net_CR': net_cr,
        'Gross_SR': gross_sr, 'Gross_CR': gross_cr, 'Gross_PnL': gross_cum,
        'Av_Trades': avg_trades, 'Score_Stat': stat
    }

def get_total_score(daily_combined):
    score = 0
    for q_start, q_end in IN_SAMPLE_QUARTERS:
        stats = calculate_quarter_stats(daily_combined, q_start, q_end)
        if stats: score += stats['Score_Stat']
    return score

def get_all_candidates(df_data, asset_name, grid_s1, grid_s2_list, asset_conf):
    candidates = []

    # 1. Breakout
    print(f"Generating {asset_name} Breakout candidates...")
    combos_s1 = list(itertools.product(grid_s1['window'], grid_s1['vol_threshold'], grid_s1['session']))
    for w, v, sess in combos_s1:
        p = {'window': w, 'vol_threshold': v, 'session': sess}
        res = run_strategy_engine(df_data, asset_name, p, asset_conf, 'breakout')
        if res is not None: candidates.append({'type': 'breakout', 'params': p, 'df': res})

    # 2. EMA
    print(f"Generating {asset_name} EMA+RSI candidates...")
    for p in grid_s2_list:
        res = run_strategy_engine(df_data, asset_name, p, asset_conf, 'ema_rsi')
        if res is not None: candidates.append({'type': 'ema_rsi', 'params': p, 'df': res})

    return candidates

def optimize_global_mix(df_data):
    print("\n" + "="*50)
    print(" STARTING GLOBAL OPTIMIZATION")
    print("="*50)

    xau_opts = get_all_candidates(df_data, 'XAU', GRID_S1_XAU, GRID_S2_LIST, ASSETS_CONFIG['XAU'])
    xag_opts = get_all_candidates(df_data, 'XAG', GRID_S1_XAG, GRID_S2_LIST, ASSETS_CONFIG['XAG'])

    print(f"\nEvaluating {len(xau_opts) * len(xag_opts)} Portfolio Combinations...")

    best_score = -9999
    best_combo = {}

    counter = 0
    total = len(xau_opts) * len(xag_opts)

    for xau in xau_opts:
        for xag in xag_opts:
            port = xau['df'].add(xag['df'], fill_value=0)
            score = get_total_score(port)

            if score > best_score:
                best_score = score
                best_combo = {'XAU': xau, 'XAG': xag}
                # print(f"New Best: {best_score:.4f}")

            counter += 1
            if counter % 500 == 0: print(f"Progress: {counter}/{total}...")

    return best_score, best_combo

# ==========================================
# 7. MAIN EXECUTION
# ==========================================

if __name__ == "__main__":
    df_data = load_data_group2(DATA_PATH)

    if not df_data.empty:
        score, combo = optimize_global_mix(df_data)

        print("\n" + "="*50)
        print(" WINNING PORTFOLIO CONFIGURATION")
        print("="*50)
        print(f" MAX SCORE: {score:.4f}")
        print("-" * 30)
        print(f" GOLD STRATEGY   : {combo['XAU']['type'].upper()}")
        print(f" GOLD PARAMETERS : {combo['XAU']['params']}")
        print("-" * 30)
        print(f" SILVER STRATEGY : {combo['XAG']['type'].upper()}")
        print(f" SILVER PARAMETERS: {combo['XAG']['params']}")
        print("="*50)

        # Report
        r_xau = run_strategy_engine(df_data, 'XAU', combo['XAU']['params'], ASSETS_CONFIG['XAU'], combo['XAU']['type'])
        r_xag = run_strategy_engine(df_data, 'XAG', combo['XAG']['params'], ASSETS_CONFIG['XAG'], combo['XAG']['type'])

        final_port = r_xau.add(r_xag, fill_value=0)
        final_rows = []
        for q_start, q_end in IN_SAMPLE_QUARTERS:
            s = calculate_quarter_stats(final_port, q_start, q_end)
            if s: final_rows.append(s)

        final_df = pd.DataFrame(final_rows)
        cols = ['Quarter', 'Gross_SR', 'Net_SR', 'Gross_CR', 'Net_CR', 'Gross_PnL', 'Net_PnL', 'Av_Trades', 'Score_Stat']
        final_df = final_df[cols]

        print(final_df.to_string(index=False))
        # final_df.to_csv("final_optimized_results.csv", index=False)

Loading 7 files...

 STARTING GLOBAL OPTIMIZATION
Generating XAU Breakout candidates...
Generating XAU EMA+RSI candidates...
Generating XAG Breakout candidates...
Generating XAG EMA+RSI candidates...

Evaluating 1296 Portfolio Combinations...
Progress: 500/1296...
Progress: 1000/1296...

 WINNING PORTFOLIO CONFIGURATION
 MAX SCORE: 18.7300
------------------------------
 GOLD STRATEGY   : BREAKOUT
 GOLD PARAMETERS : {'window': 288, 'vol_threshold': 0.5, 'session': 'full'}
------------------------------
 SILVER STRATEGY : BREAKOUT
 SILVER PARAMETERS: {'window': 144, 'vol_threshold': 0.05, 'session': 'us_only'}
Quarter  Gross_SR    Net_SR  Gross_CR    Net_CR  Gross_PnL  Net_PnL  Av_Trades  Score_Stat
2023-01  0.027477 -0.250520  0.048967 -0.418786      212.5  -1947.5   1.617978   -0.500257
2023-07  1.993005  1.535439  6.839562  4.771139     7290.9   5620.9   1.217391    1.787677
2023-10  2.237860  1.894607  8.207757  6.815706    13261.9  11241.9   1.478261    3.374458
2024-04  0.196748 -